# Algorithmes de recherche de chemins

## Objet du problème


### Données
- Un ensemble fini $X$, l'ensemble des *actions*.
- Un ensemble fini $S$, l'ensemble des *états*, et $\delta:S\times X\rightarrow S$ une fonction partielle dans le sens où, pour tout $p\in S$, et $x\in X$, $q=\delta(p,x)$ n'est défini que si $x$ appartient à une partie $X_p$ de $X$. On dit que $q$, noté aussi $p.x$, est l'état obtenu en *exécutant* l'action $x$ sur l'état $p$. $X_p$ est l'ensemble des actions *légales* sur $p$.
- Un *état initial* $d\in S$ et un ensemble d'*états finaux* $F\subset S$

On introduit alors le graphe orienté $G=(S,A)$ avec $A=\{(p,p.x)\,|\,p\in S, x\in X_p\}$.  
### Objectif
Trouver, s'il en existe, un  chemin de $G$ partant de l'état initial et aboutissant à un état final (chemin *réussi*).
Plus précisément, l'algorithme doit retourner la liste $(x_1,\ldots,x_n)$ des actions à éxecuter à partir de l'état initial pour suivre le chemin réussi trouvé :  
$d.x_1.x_2.\ldots.x_n\in F$.

On propose l'algorithme suivant

## Parcours en profondeur (DFS)


On écrit une fonction générale
```python
dfs(start_state,
    is_end_state,  # state -> bool
    legal_actions, # state -> tuple of actions
    delta          # state, action -> state
    )
```
dont les arguments se comprennent aisément :
- `start_state` $\in S$ est l'état initial ;
- `is_end_state` $:p\in S\mapsto(p\in F)\in B=\{\text{True},\text{False}\}$ ,
- `legal_actions` $:p\in S\mapsto X_p$ où $X_p$ doit être représenté par un tuple d'actions ;
- `delta` $:p\in S,x\in X_p\mapsto p.x\in S$.
- `is_blocked` $:S\rightarrow B$, `is_blocked` $(p)$ renvoie True si on sait dés le départ qu'il n'existe aucun chemin d'origine $p$ et d'extrémité un état final ;  

et qui renvoie une liste d'actions correspondant à un chemin réussi s'il en existe, et sinon déclenche une exception.

Noter que les états doivent être représentés par des objets python hashables (typiquement un tuple d'objets hashables est hashable, pas une liste)

In [ ]:
def dfs(start_state, is_end_state, legal_actions, delta, is_blocked = lambda state: False):

    mark = set()

    def search(state, actions):
        h = hash(state)
        if h not in mark and not is_blocked(state):
            if is_end_state(state):
                raise Exception(actions)
            mark.add(h)
            for action in legal_actions(state):
                newState = delta(state, action)
                newActions = actions + [action]
                search(newState, newActions)

    try:
        search(start_state, [])
    except Exception as actions:
        return actions.args[0]

    raise Exception("Il n'existe pas de chemin réussi")

De manière informelle, le parcours consiste à suivre des chemins les plus longs (profonds) possibles puis, si l'on n'a pas rencontré d'état final, à les remonter pour suivre des branches non encore explorées.

On peut prouver beaucoup de propriétés intéressantes du parcours en profondeur. Ici, il nous suffit de démontrer quelques faits (rappelons que pour montrer une propriété relative à une fonction récursive, il suffit de prouver que si cette propriété est vérifiée par les sous appels, alors elle l'est pour l'appel de la fonction).

- Pour tout couple $(p,u)$ argument de la fonction récursive *search*, la liste d'actions $u$ définit un chemin de l'état initial à $p$. Cela montre la correction de la fonction *dfs* dans le cas où elle renvoie un objet. On suppose dorénavant que l'on n'est pas dans ce cas.
- Un état $p$ (en fait *hash* $(p)$), appartient à l'ensemble *mark* si et seulement si la fonction *search* a déjà été exécutée avec l'argument $p$. De plus *search* $(p, \ldots)$ est exécuté au plus une fois. En effet, on vérifie aisément que cette propriété est un invariant de *search*. Comme il n'y a qu'un nombre fini d'états, cela montre au passage l'arrêt de la fonction *search*.
- On peut  aussi montrer que la fonction *search* satisfait les spécifications suivantes :  
*search* $(p,u)$ ajoute à l'ensemble *mark* tous les états $q$ pour lesquels, au moment de l'appel  *search* $(p,u)$, il existe un chemin de $p$ à $q$ formé d'états absents de *mark*.  
Puisque au départ, *mark* $=\emptyset$, cela prouve que l'appel principal *search* $(d,())$ explore tous les sommets accessibles depuis $d$ et qu'aucun d'eux n'est final. CQFD

### Application

[solitaire](../solitaire/solitaire.ipynb)

## Parcours en largeur (BFS)


Il s'agit d'un algorithme qui, en théorie, donne le même résultat que le parcours en profondeur, mais avec deux différences notables :
- Il est beaucoup moins efficace ; les temps de calcul sont rédhibitoires si les états finaux sont tous éloignés de l'état initial.
- Par contre, quand il fournit un chemin réussi, celui-ci est de *longueur* (nombre d'arcs le constituant) minimale, on dit que le chemin est *optimal*.

L'idée est de parcourir les états dans l'ordre de leur *distance* à l'état initial (distance $(d,p)=$  min $\{$ longueur $(\gamma)$ | $\gamma$ chemin de $d$ à $p$ $\}$). On parcourt les états à la distance 1, puis les états à la distance 2, etc. On voit bien que, si par exemple les états finaux sont à une distance maximale de $d$, il faudra parcourir pratiquement tous les états avant de tomber sur eux ; d'où l'inefficacité dans le cas d'un grand graphe.

Par contre, s'il n'y a pas trop d'états, le parcours en largeur va permettre de calculer efficacement, pour TOUS les états $p$, la distance de $d$ à $p$ et un chemin optimal de $d$ à $p$, sauf si $p$ n'est pas accessible depuis $p$, auquel cas distance $(d,p)=\infty$. C'est ce que l'on va faire ici (en oubliant donc la notion d'état final).

Voici un parcours en largeur basique prenant en entrée l'état de départ $d$ et les fonctions $\alpha: p\mapsto X_p$ et $\delta:(p,x)\mapsto p.x$.

bfs $(d,\alpha,\delta)$:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$M\leftarrow\emptyset$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$Q\leftarrow$ une file FIFO (premier entré, premier sorti) vide  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**définir** *traiter* $(p)$ :  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Effectuer une certaine *action* prédéfinie sur l'état $p$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ajouter $p$ à $M$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*push* $(p,Q)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*traiter* $(d)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**tant que** $Q\neq\emptyset$ **faire**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p\leftarrow$ *pop* $(Q)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**pour tout** $x\in\alpha(p)$ **faire**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$q\leftarrow\delta(p,x)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** $q\not\in M$ **alors**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*traiter* $(q)$

### Ordre de parcours

Comme un état  $p$ n’est *traité* que s’il n’appartient pas à $M$ et que *traiter* $(p)$ ajoute l'état $p$ à $M$, on voit déjà que *traiter* $(p)$ (ou  *action* $(p)$)  est exécuté au plus une fois pour chaque état. Notons $T$ l'ensemble des états traités ; au début du parcours, $T=\emptyset$.

On démontre que, à la fin du parcours,  $T=$ *acc* $(d)$ l'ensemble des états accessibles depuis $d$. Pour cela, il suffit de vérifier que la boucle principale satisfait l’invariant suivant
:
- $d\in T\subset$ *acc* $(d)$
- $\forall p,q, (p\in T\setminus Q$ et $(p,q)\in A) \Rightarrow q\in T$

Pour $k\in\mathbf N$, notons $S_k=\{p\in S$ | distance($d,p$) $=k\}$ et soit $k_{\text{max}}$  la plus grande valeur de $k$ pour laquelle $S_k$ est non vide. On voit par récurrence que, pour tout $k\leqslant k_{\text{max}}$, il existe une étape de l’algorithme où $T=\cup_{i\leqslant k}S_i$ et $Q=S_k$. Cela montre bien que $d$ est traité, puis les états de $S_1$, puis ceux de $S_2$, etc.

### Complexité

Les états ajoutés à $Q$ durant le déroulement de l’algorithme sont ceux de *acc* $(d)$ et chacun
de ces états est ajouté à $Q$ et donc aussi supprimé de $Q$ une unique fois. La boucle **tant que** est donc exécutée Card(*acc* $(d)$) $\leqslant$ Card $(S)$ fois (ce qui, au passage, fournit la preuve d’arrêt), le $p$ du corps de la boucle décrivant l’ensemble *acc* $(d)$. De plus, le corps de la boucle **pour tout** est exécuté Card($p.\alpha(p)$) fois pour chaque $p\in$ *acc* $(d)$. Au total, la dernière ligne est donc exécutée moins de $\Sigma_{p\in S}$ Card($p.\alpha(p)$) = Card($A$) fois, ce qui prouve que, si la complexité de *action* est O($1$), la complexité du parcours est O(Card($S$) $+$ Card($A$))

### Implémentation

On introduit un dictionnaire *dico* et le traitement (ou *process*) d'un état $p$ consiste à définir *dico*[$p$] $=(x, q, k)$ où $k=$ distance $(d,p)$,  $q$ est l'avant dernier état d'un chemin optimal de $d$ à $p$ et $q.x=p$ ($x,q=$ *None*,*None* si $p=d$). Ainsi $M=\{p$ | dico[$p$] est défini $\}$ et *dico* permet de retrouver, pour tout $p$, un chemin optimal de $d$ à $p$.

On écrit donc une fonction
```python
bfs(start_state, legal_actions, delta)
```
qui renvoie un couple de fonctions (*dist*,*path*) avec, pour tout état $p$
- *dist*($p$) $=k=$ distance $(d,p)$ ou *None* si $p$ n'est pas accessible depuis $d$
- *path*($p$) $=$ [$x_1,\ldots,x_k$] avec $d.x_1.x_2\ldots x_k=p$ ou *None*

In [ ]:
def bfs(start_state,
        legal_actions, # state -> tuple of actions
        delta          # state, action -> state
        ):

    from queue import Queue # FIFO

    q = Queue()
    dico = dict()

    def process(state, action, predState, d):
        #print(d, end = '\r')
        dico[state] = (action, predState, d)
        q.put(state)

    process(start_state, None, None, 0)
    while not q.empty():
        state = q.get()
        _, _, d = dico[state]
        for action in legal_actions(state):
            newState = delta(state, action)
            newAction = action
            if newState not in dico:
                process(newState, newAction, state, d + 1)

    def distance(state):
        return None if state not in dico else dico[state][2]

    def path(state):
        if state in dico:
            l = []
            action, s, _ = dico[state]
            while action is not None:
                l.append(action)
                action, s, _ = dico[s]
            l.reverse()
            return l
        else:
            return None

    return distance, path

### Test
$G=(S,A)$ avec $S=\{0,\ldots,9\}$ et $A=\{(0,3),$ $(0,2),$ $(0,1),$ $(2,6),$ $(2,5),$ $(3,6),$ $(4,5),$ $(4,7),$ $(4,1),$ $(4,2),$ $(5,7),$ $(5,9),$ $(5,8),$ $(6,9),$ $(6,4),$ $(7,8),$ $(7,4),$ $(9,4),$ $(9,8)\}$

In [ ]:
g = [[3,2,1],[],[6,5],[6],[5,7,1,2],[7,9,8],[9,4],[8,4],[],[4,8]]
dist, path = bfs(0, lambda p: g[p], lambda _, q: q)
resultats = ''
for p in range(10):
    resultats += f"l'état {p} est à la distance {dist(p)} de 0 et un chemin optimal est {[0] + path(p)}\n"
#print(resultats)


## Algorithme A*

### Données
En plus des données $S,X,\delta,d,F$, on suppose que chaque action $x\in X$ est munie d'un *coût* $w(x)\in\mathbf R_+$.  
On considére alors que chaque chemin $(p,p.x_1,p.x_1.x_2,\dots)$ admet lui aussi un coût $w(x_1)+w(x_2)+\ldots$.  

### Objectif
On cherche un chemin réussi de coût minimum (chemin *optimal*), encore une fois renvoyé sous la forme d'une liste d'actions.  
Ce coût minimum est noté $w_{\text{min}}(d,F)$. S'il n'y a pas de chemin réussi, on convient que $w_{\text{min}}(d,F)=\infty$.  

### L'algorithme
En plus des données précédentes, l'algorithme A* utilise une fonction *heuristique* $h:S\rightarrow \mathbf R_+$ qui est une sous-estimation de $h_0(p)=w_{\text{min}}(p,F)$, c'est à dire    
$\forall p\in S,\,h(p)\leqslant h_0(p)=w_{\text{min}}(p,F)$  

On introduit aussi le *coût* d'un état $c_0(p)=w_{\text{min}}(d,p)$ de sorte que, si $p$ est un état participant à un chemin optimum, $f_0(p)=w_{\text{min}}(d,F)=c_0(p)+h_0(p)$.

Dans l'algorithme A* :
- frontiere est un ensemble de couples $(p, u)$ où $p\in S$ et $u=(x_1,\ldots,x_n)\in X^n$ est un tuple d'actions permettant de passer de l'état de départ à l'état $p$ : $d.x_1.x_2.\ldots.x_n=p$.
- $f$ est la fonction *priorité* ; quand un couple $(p,u)$ est ajouté à frontiere, $f(p)=w(u)+h(p)$ et coutActuel $(p)=w(u)\geqslant c_0(p)$.

On traite en priorité un couple $(p,u)\in$ frontiere si $f(p)$ est minimum.

A*($d,F,X,\delta,h,w$) :  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;frontiere $\leftarrow\{(d,())\}$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$f(d)\leftarrow h(d)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;coutActuel $(d)\leftarrow0$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**tant que** frontiere $\neq\empty$ **faire**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;supprimer de frontiere un élément $(p,\text{actions})$ tel que $f(p)$ minimum   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** $p\in F$ **alors**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**renvoyer** actions  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**pour** action $\in X_p$ **faire**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$p'\leftarrow \delta(p,\text{action})$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\text{actions}'\leftarrow$ actions auquel on ajoute action  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\text{cout}'\leftarrow w(\text{actions}')$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**si** coutActuel $(p')$ n'est pas défini ou est $>$ à $\text{cout}'$ **faire**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;coutActuel $(p')\leftarrow \text{cout}'$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$f(p')\leftarrow \text{cout}'+h(p')$   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ajouter $(p',\text{actions}')$ à frontiere et supprimer éventuellement de frontiere un ancien $(p',\ldots)$    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**faillir**

L'algorithme faillit s'il n'existe pas de chemin réussi, et sinon il renvoie un chemin optimal.

La preuve de terminaison et de validité de A* est assez difficile. On trouve sur internet beaucoup de preuves plus ou moins correctes, plus ou moins claires etc. La seule démonstration correcte et complète que j'ai pu trouver est celle des concepteurs de l'algorithme
[A Formal Basis for the Heuristic Determination of Minimum Cost Paths](https://web.archive.org/web/20160322055823/http://ai.stanford.edu/~nilsson/OnlinePubs-Nils/PublishedPapers/astar.pdf).

### Implémentation

On écrit une fonction `aStar` dont voici les arguments (les états doivent être hashables) :
- `start_state` $\in S$ l'état initial ;
- `is_end_state` $:p\in S\mapsto(p\in F)\in B=\{\text{True},\text{False}\}$ ;
- `legal_actions` $:p\in S\mapsto X_p$ où $X_p$ doit être représenté par un tuple d'actions ;
- `delta` $:p\in S,x\in X_p\mapsto p.x\in S$ ;
- `is_blocked` $:S\rightarrow B$, `is_blocked` $(p)$ renvoie True si on sait dés le départ qu'il n'existe aucun chemin d'origine $p$ et d'extrémité un état final ;
- `heuristic` $:S\rightarrow$ `float` ou `int` ;
- `cost` $:[x_1,\ldots,x_n]\in X^n\mapsto w(x_1)+\ldots +w(x_n)\in$ `float` ou `int`.  

La fonction renvoie une liste d'actions ou, si elle faillit, déclenche une exception.


On représente frontiere par une [file de priorité](https://fr.wikipedia.org/wiki/File_de_priorit%C3%A9) de type [`queue.PriorityQueue`](https://docs.python.org/3/library/queue.html#queue.PriorityQueue)

In [ ]:
def is_blocked_default(state):
    return False

def heuristic_default(state):
    return 0

def cost_default(actions):
    return len(actions)

def aStar(start_state,
          is_end_state,                    # state -> bool
          legal_actions,                   # state -> tuple of actions
          delta,                           # state, action -> state
          is_blocked = is_blocked_default, # state -> bool
          heuristic = heuristic_default,   # state -> numeric
          cost = cost_default              # actions_list -> numeric
    ):

    from queue import PriorityQueue

    frontier = PriorityQueue()
    h = heuristic(start_state)
    frontier.put((h, start_state, []))
    currentCost = {start_state : 0}
    while not frontier.empty():
        _, state, state_action = frontier.get()
        if currentCost[state] == cost(state_action): # dans le cas contraire, state aurait dû être supprimé (voir (1))
            if is_end_state(state):
                return state_action
            for action in legal_actions(state):
                newState = delta(state, action)
                if not is_blocked(newState):
                    newState_action = state_action + [action]
                    newCost = cost(newState_action)
                    if newState not in currentCost or newCost < currentCost[newState]:
                        currentCost[newState] = newCost
                        f = heuristic(newState) + newCost
                        frontier.put((f, newState, newState_action)) # (1) on devrait supprimer de frontier une éventuelle ancienne occurence de newState
    raise Exception("Aucun état final n'est accessible depuis l'état initial")


### Applications

[8-puzzle](../8-puzzle/8-puzzle.ipynb)

[sokoban](../sokoban/sokoban.ipynb)